# How to Use Inference Endpoints to Embed Documents

## Introduction

If we have a dataset that we want to embed for semantic search (or QA, RAG, etc), what would be the easiest way to embed this and put it in a new dataset?

In this example, we will deploy the [`jinaai/jina-embeddings-v2-base-en`](https://huggingface.co/jinaai/jina-embeddings-v2-base-en) embedding model using the [Inference Endpoint](https://huggingface.co/inference-endpoints/dedicated) to save time and money.

In [ ]:
!pip install -qU aiohttp==3.8.3 datasets==2.14.6 pandas==1.5.3 requests==2.31.0 tqdm==4.66.1 huggingface-hub>=0.20

## Setups

In [ ]:
import asyncio
from getpass import getpass
import json
from pathlib import Path
import time
from typing import Optional

from aiohttp import ClientSession, ClientTimeout
from datasets import load_dataset, Dataset, DatasetDict
from huggingface_hub import (
    notebook_login, create_inference_endpoint, list_inference_endpoints, whoami
)
import numpy as np
import pandas as pd
import requests
from tqdm.auto import tqdm

In [ ]:
notebook_login()

In [ ]:
# config parameters
DATASET_IN = "derek-thomas/dataset-creator-reddit-bestofredditorupdates"
DATASET_OUT = "processed-subset-bestofredditorupdates"
ENDPOINT_NAME = "boru-jina-embeddings-demo-ie"

# number of async workers to use
MAX_WORKERS = 5
# None to use all
ROW_COUNT = 100

`DATASET_IN` is where our text data is, and `DATASET_OUT` is where our embeddings will be stored.

Inference Endpoints offers a number of GPUs that we can choose from.

In [ ]:
# GPU choice
VENDOR = 'aws'
REGION = 'us-east-1'
INSTANCE_SIZE = 'x1'
INSTANCE_TYPE = 'nvidia-a10g'

In [ ]:
who = whoami()
organization = getpass(
    prompt="What is your HuggingFace username or organization? (with an added payment method)"
)

namespace = organization or who["name"]

## Get dataset

In [ ]:
dataset = load_dataset(DATASET_IN)
dataset

In [ ]:
dataset['train']

In [ ]:
documents = dataset['train'].to_pandas().to_dict('records')[:ROW_COUNT]
len(documents), documents[0]

## Inference Endpoints

### Create Inference Endpoint

We will use the API to create an Inference Endpoint

In [ ]:
try:
    endpoint = create_inference_endpoint(
        ENDPOINT_NAME,
        repository='jinaai/jina-embeddings-v2-base-en',
        revision="7302ac470bed880590f9344bfeee32ff8722d0e5",
        task='sentence-embeddings',
        framework='pytorch',
        accelerator='gpu',
        instance_size=INSTANCE_SIZE,
        instance_type=INSTANCE_TYPE,
        region=REGION,
        vendor=VENDOR,
        namespace=namespace,
        custom_image={
            'health_route': '/health',
            'env': {
                'MAX_BATCH_TOKENS': str(MAX_WORKERS * 2048),
                'MAX_CONCURRENT_REQUESTS': "512",
                'MODEL_ID': "/repository"
            },
            'url': 'ghcr.io/huggingface/text-embeddings-inference:0.5.0'
        },
        type='protected'
    )
except:
    endpoint = [
        ie for ie in list_inference_endpoints(namespace=namespace)
        if ie.name == ENDPOINT_NAME
    ][0]
    print(f"Using existing endpoint {endpoint.name}")

Note:
* We used `jinaai/jina-embeddings-v2-base-en` as our model. For reproducibility we pinned it to a specific version
* Most embedding models are based on BERT architecture.
* `MAX_BATCH_TOKENS` is chosen based on our number of workers and the context window of our embedding model.
* `type="protected"` utilizes the security from Inference Endpoints
* We used 1x Nivida A10 GPU since `jina-embeddings-v2` is memory hungry

In [ ]:
%%time
endpoint.wait()

When we use `endpoint.client.post` we get a bytes string back. We need to convert this to a `np.array`:

In [ ]:
response = endpoint.client.post(
    json={
        'inputs': "This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music!",
        'truncate': True
    },
    task='feature-extraction'
)
response = np.array(json.loads(response.decode()))
response[0]

We may have inputs that exceed the context. In such scenarios, it is up to us to handle them. In this example, we would truncate rather than have an error:

In [ ]:
embedding_input = "This input will get multiplied" * 10000
print(f"The length of the `embedding_input` is: {len(embedding_input)}")

response = endpoint.client.post(
    json={
        'inputs': embedding_input,
        'truncate': True
    },
    task='feature-extraction'
)
response = np.array(json.loads(response.decode()))
print(response[0])

### Get embeddings

In [ ]:
async def request(document, semaphore):
    # semaphore guard
    async with semaphore:
        result = await endpoint.async_client.post(
            json={
                'inputs': document['content'],
                'truncate': True
            },
            task='feature-extraction'
        )
        result = np.array(json.loads(result.decode()))
        document['embedding'] = result[0] # assume API's output can be directly assigned
        return document


async def main(documents):
    # Semaphore to limit concurrent requests
    semaphore = asyncio.BoundedSemaphore(MAX_WORKERS)

    tasks = [request(document, semaphore) for document in documents]

    for f in tqdm(asyncio.as_completed(tasks), total=len(documents)):
        await f

In [ ]:
start = time.perf_counter()

# get embeddings
await main(documents)

# make sure we got it all
count = 0
for document in documents:
    if 'embedding' in document.keys() and len(document['embedding']) == 768:
        count += 1
print(f"Embeddings = {count} documents = {len(documents)}")


elapsed_time = time.perf_counter() - start
minutes, seconds = divmod(elapsed_time, 60)
print(f"{int(minutes) min {seconds:.2f}} sec")

### Pause Inference Endpoint

If we finish running, we can pause the endpoint so we do not incur any extra charges and also analyze the cost:

In [ ]:
endpoint = endpoint.pause()
print(f"Endpint status: {endpoint.status}")

### Push updated dataset to Hub

We now have our documents updated with the embeddings we wanted. We need to first convert it back to a `Dataset` format:

In [ ]:
df = pd.DataFrame(documents)
dataset = DatasetDict({'train': Dataset.from_pandas(df)})

In [ ]:
dataset.push_to_hub(repo_id=DATASET_OUT)
print(f'Dataset is at https://huggingface.co/datasets/{who["name"]}/{DATASET_OUT}')

### Analyze usage

Go to our `dashboard_url`

In [ ]:
dashboard_url = f"https://ui.endpoints.huggingface.co/{namespace}/endpoints/{ENDPOINT_NAME}"
print(dashboard_url)

### Deleta endpoint

We can delete our endpoint if we do not need it anymore:

In [ ]:
endpoint = endpoint.delete()

if not endpoint:
    print("Endpoint deleted successfully")
else:
    print("Delete endpoint manually")